In [1]:
import datetime
import glob
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [ ]:
df_CPI = pd.read_csv('../final_databases/no_mask/all_campaigns.csv')

In [ ]:
date = df_CPI['filename'].str.split('_')
dates=[]
campaigns = {'ARM': 2000, 'CRYSTAL_FACE_UND':2002,  'CRYSTAL_FACE_NASA':2002, 'MIDCIX':2004, 'ICE_L':2007,\
        'OLYMPEX':2015, 'MPACE':2004}
campaigns = {'CRYSTAL_FACE_NASA':2002}
for campaign in campaigns:
    year = campaigns[campaign]
    if campaign == 'OLYMPEX'and dat[0] == 11 or dat[0] == 12:
        year = 2014
    if campaign == 'AIRS_II' and dat[0] == 11 or dat[0] == 12:
        year = 2003
    for dat in date:
        dat=str(year)+dat[0]+dat[1]
        dates.append(dat)

In [ ]:
df_CPI['date'] = dates
df_CPI['date'] = pd.to_datetime(df_CPI['date'], format='%Y%m%d-%H%M%S%f')
cols = list(df_CPI)
# move the column to head of list using index, pop and insert
cols.insert(2, cols.pop(cols.index('date')))
cols

df_CPI = df_CPI.loc[:, cols]
df_CPI

In [ ]:
df_CPI['date'].groupby([df_CPI['date'].dt.month, df_CPI['date'].dt.day]).agg('count')

In [ ]:
df_CPI = df_CPI.sort_values(by="date")

In [ ]:
g =df_CPI['date'].groupby([(df_CPI['date'].dt.day)])
df_CPI['date'].head()

In [ ]:
df_CPI.to_csv('final_databases/no_mask/all_campaigns_datesorted.csv', index=False)